In [21]:
import os

In [22]:
%pwd

'/home/muhammed-shafeeh/AI_ML/waste_detection_yolo_v5'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/muhammed-shafeeh/AI_ML/waste_detection_yolo_v5'

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    feature_store_root_dir: Path
    data_yaml_file: str
    pretrained_model_path: Path
    pretrained_model_name: str
    params_epochs: int
    params_batch_size: int
    
    

In [68]:
from src.yoloproject.constants import *
from src.yoloproject.utils.common import read_yaml, create_directories
from yoloproject.utils.common import read_yaml_dict

In [69]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_validation.root_dir])
        
    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        
        create_directories([config.root_dir, config.feature_store_root_dir])
        
        model_training_config = ModelTrainingConfig(
            root_dir=Path(config.root_dir),
            feature_store_root_dir=Path(config.feature_store_root_dir),
            data_yaml_file=config.data_yaml_file,  
            pretrained_model_path=Path(config.pretrained_model_path),
            pretrained_model_name=config.pretrained_model_name,
            params_epochs=self.params.EPOCHS,
            params_batch_size=self.params.BATCH_SIZE, 
        )
        
        return model_training_config

In [70]:
import yaml
import os
from pathlib import Path


In [71]:

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
        
    def model_training(self):
        '''
        Train the YOLOv5 model using the provided data and parameters
        '''
        # Perform the necessary training steps
        try:
            with open(self.config.feature_store_root_dir / self.config.data_yaml_file, "r") as stream:
                num_class = str(yaml.safe_load(stream)["nc"])
                
            #model_config_file_name = self.config.pretrained_model_name.split(".")[0]
            model_config_file_name = self.config.pretrained_model_name.split("/")[-1].split(".")[0]
            print(model_config_file_name)
            
            config = read_yaml_dict(Path(f'yolov5/models/{model_config_file_name}.yaml'))
            config['nc'] = int(num_class)
            
            with open(f'yolov5/models/custome_{model_config_file_name}.yaml', "w") as file:
                yaml.dump(config, file)
            
            # Train the model using the provided parameters
            data_yaml = Path(self.config.feature_store_root_dir) / self.config.data_yaml_file
            data_yaml = data_yaml.resolve()  # Get absolute path

            os.system(f"cd yolov5/ && python train.py --img 640 --batch {self.config.params_batch_size} --epochs {self.config.params_epochs} --data {data_yaml} --cfg ./models/custome_yolov5s.yaml --weights {self.config.pretrained_model_name} --name yolov5_results --cache")
            os.system(f"cp yolov5/runs/train/yolov5_results/weights/best.pt {self.config.root_dir}")
            
            os.system("rm -rf yolov5/runs")
        
        except Exception as e:
            print(f"Error while building the model: {e}")

model_training:
  root_dir: artifacts/model_training
  feature_store_root_dir: artifacts/data_ingestion/feature_store/
  data_yaml_file: data.yaml
  pretrained_model_path: yolov5/models
  pretrained_model_name: yolov5s.pt

In [72]:
config_manager = ConfigurationManager()
        
# Get Model Training Configuration
model_training_config = config_manager.get_model_training_config()

# Initialize Model Training Class
trainer = ModelTraining(config=model_training_config)

# Run the Training
print("Starting YOLOv5 Training...")
trainer.model_training()
print("Training Completed Successfully!")

Starting YOLOv5 Training...
yolov5s


train: weights=yolov5s.pt, cfg=./models/custome_yolov5s.yaml, data=/home/muhammed-shafeeh/AI_ML/waste_detection_yolo_v5/artifacts/data_ingestion/feature_store/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.6.0+cu124 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay

Training Completed Successfully!
